# Optuna

- https://habr.com/ru/articles/704432/
- https://optuna.readthedocs.io

In [ ]:
%pip install optuna pandas plotly nbformat scikit-learn

## Базовый пример: оптимизация функции

In [ ]:
import optuna

def objective(trial):
    x = trial.suggest_float(name="x", low=-10, high=10, log=False)
    return (x - 2) ** 2

study = optuna.create_study(study_name="experiment 1")

# Минимизация функции objective

study.optimize(objective, n_trials=100, timeout=300)

study.best_params

## Различные типы параметров и направление оптимизации

In [ ]:
def objective(x, y, z):
    match x:
        case "A":
            return (y * z) ** 2
        case "B":
            return (y * z) ** 4

def wrapper(trial):
    x = trial.suggest_categorical(name="x", choices=["A", "B"])
    y = trial.suggest_float(name="y", low=-10, high=10, step=0.1, log=False)
    z = trial.suggest_int(name="z", low=1, high=10, step=1, log=False) 
    
    return objective(x, y, z)

study = optuna.create_study(study_name="experiment 2", direction="maximize")
study.optimize(wrapper, n_trials=100, timeout=300)

study.best_params, study.best_value, study.best_trial

## Поиск по двум критериям

In [ ]:
import time

def objective(x, y, z):
    match x:
        case "A":
            return (y * z) ** 2
        case "B":
            return (y * z) ** 4

def wrapper(trial):
    x = trial.suggest_categorical(name="x", choices=["A", "B"])
    y = trial.suggest_float(name="y", low=-10, high=10, step=0.1, log=False)
    z = trial.suggest_int(name="z", low=1, high=10, step=1, log=False) 
    
    t1 = time.perf_counter()
    result = objective(x, y, z)
    t2 = time.perf_counter()

    return result, t2 - t1

study = optuna.create_study(study_name="experiment 3", directions=["maximize", "minimize"])
study.optimize(wrapper, n_trials=100, timeout=10)

# Многокритериальный подбор параметров не предполагает наличие оптимальных параметров best_params.
# RuntimeError: A single best trial cannot be retrieved from a multi-objective study.
# Consider using Study.best_trials to retrieve a list containing the best trials.

optuna.visualization.plot_pareto_front(study, target_names=["Значение функции", "Время выполнения"])

### Анализ испытаний по границе Парето

In [ ]:
print(f"Число комбинаций гиперпараметров по границе Парето: {len(study.best_trials)}")

trial_least_time = min(study.best_trials, key=lambda t: t.values[1])

print("Комбинация гиперпараметров с наименьшим временем выполнения: ")
print(f"\tnumber: {trial_least_time.number}")
print(f"\tparams: {trial_least_time.params}")
print(f"\tvalues: {trial_least_time.values}")

trial_with_highest_value = max(study.best_trials, key=lambda t: t.values[1])

print("Комбинация гиперпараметров с наибольшим значением: ")
print(f"\tnumber: {trial_with_highest_value.number}")
print(f"\tparams: {trial_with_highest_value.params}")
print(f"\tvalues: {trial_with_highest_value.values}")

### Анализ значимости гиперпараметров

In [ ]:
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.values[0], target_name="???"
)

## Преобразование и сохранение результатов эксперимента

In [ ]:
df = study.trials_dataframe()
df.to_csv('study.csv')

df

In [ ]:
import pandas as pd

loaded = pd.read_csv('study.csv')

loaded

In [ ]:
import joblib

joblib.dump(study, 'experiments.pkl')
study_loaded = joblib.load('experiments.pkl')
study_loaded.trials_dataframe()